In [ ]:
#hide
#default_exp misc
#default_cls_lvl 3
from nbdev.showdoc import show_doc

# sniptool.misc

> magic commands for snippets management, supported by a sqlite db.

In [ ]:
# exporti
import os
import datetime
import pandas as pd
from glob import glob
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic
import requests
from os.path import expanduser

In [ ]:
# exporti
@magics_class
class MiscMagics(Magics):
    """snip magic for snippets management.
    Provides the %snip magic.
    """

    @line_magic
    def slack(self, line):
        "my line magic"
        def post_message_to_slack(text, blocks = None, slack_token=open(expanduser('~/.slack_token')).read().strip(), channel='local'):
            return requests.post('https://slack.com/api/chat.postMessage', {
                'token': slack_token,
                'channel': channel,
                'text': text
            }).json()

        if not line:
            line = 'done'
        post_message_to_slack(line)
# curl -X POST -H 'Authorization: Bearer ' \
# -H 'Content-type: application/json' \
# --data '{"channel":"C015RC5PU85","text":"I hope the tour went well, Mr. Wonka."}' \
# https://slack.com/api/chat.postMessage
    



    @line_magic
    def d(self, parameter_s="", n_cols=5, regex="", cap=False):
        import inspect

        def chunks(names: list, n: int):
            """Yield successive n-sized chunks from lst."""
            r = int(len(names) / n + 0.5)
            for i in range(0, len(names), r):
                yield names[i : i + r]

        def output(items, n_cols=5):
            newcolumns = list(chunks(items, n_cols))
            n_rows, n_columns = len(newcolumns[0]), len(newcolumns)
            newcolumns[-1].extend([""] * (n_rows * n_columns - len(items)))
            array_outputs = [[row[i] for row in newcolumns] for i in range(n_rows)]
            col_width = (
                max(len(word) for row in array_outputs for word in row) + 2
            )  # padding
            for row in array_outputs:
                print("".join(word.ljust(col_width) for word in row))

        opts, argsl = self.parse_options(parameter_s, "cn:s:", mode="string")
        args = argsl.split()
        if "n" in opts:
            n_cols = int(opts["n"])
        if "c" in opts:
            cap = True
        if len(args) > 1:
            regex = args[1]
        for _, f in enumerate(inspect.stack()[2:]):
            if args[0] in f[0].f_locals:
                target = f[0].f_locals[args[0]]
                items = list(filter(lambda x: x[0] != "_", dir(target)))
                if regex:
                    items = list(filter(lambda x: x.find(regex) != -1, items))
                if not cap:
                    items = list(filter(lambda x: x[0].islower(), items))
                # TODO if no external attributes (like functions)
                output(items, n_cols=n_cols)
                break

    @line_magic
    def lint(self, parameter_s=""):
        opts, argsl = self.parse_options(parameter_s, "abf", mode="string")
        args = argsl.split()
        assert os.path.isfile(args[0])
        if "f" in opts:
            cmd = f"autopep8 --in-place {args[0]}"
            print("autopep8: finished")
        elif "b" in opts:
            cmd = f"nbblack {args[0]}"
        elif "a" in opts:
            cmd = f"pylint --disable=R,C --rcfile ~/.pylintrc {args[0]}"
        else:
            cmd = f"pycodestyle --show-source --ignore E501 {args[0]}"
        print(cmd)
        os.system(cmd)

    @cell_magic
    def runingb(self, line, codes):
        from multiprocessing import Process
        from tempfile import NamedTemporaryFile

        def wrapper(pyfile):
            import subprocess
            import time

            start = time.time()
            proc = subprocess.Popen(["python", pyfile], stdout=subprocess.PIPE)
            print(proc.stdout.readline().decode())
            print("Wall time:", time.time() - start)

        f = NamedTemporaryFile(mode="w", suffix=".py", delete=False)
        #         print(f.name)
        with open(f.name, "w") as fout:
            fout.write(codes)
        p = Process(target=wrapper, args=(f.name,), name="wrapper")
        p.start()
        

    @cell_magic
    def pep8(self, line, cell, auto=False):
        """pycodestyle cell magic for pep8"""
        if cell.startswith(("!", "%%", "%")):
            return
        with tempfile.NamedTemporaryFile(mode="r+", delete=False) as f:
            f.write("# The %%pycodestyle cell magic was here\n" + cell)
            f.flush()
            f.close()
        format = "%(row)d:%(col)d: %(code)s %(text)s"
        pycodestyle = pycodestyle_module.StyleGuide(format=format)
        pcs_result = pycodestyle.check_files(paths=[f.name])
        try:
            os.remove(f.name)
        except OSError as e:  # if failed, report it back to the user ##
            logger.error("Error: %s - %s." % (e.filename, e.strerror))

#     @cell_magic
#     def toc(self, ipynb, markdown_content):
#         def mkreader(md):
#             for line in ("\n" + md).split("\n#"):
#                 if line:
#                     yield ("#" + line)

#         cells = json.load(open(ipynb))
#         for source in reversed(list(mkreader(markdown_content))):
#             cell = {
#                 "cell_type": "markdown",
#                 "id": "",
#                 "metadata": {},
#                 "source": [line + "\n" for line in source.splitlines()],
#             }
#             cells["cells"].insert(0, cell)
#         with open(ipynb, "w") as fout:
#             json.dump(cells, fout, indent=2)

#     @line_magic
#     def tidy(self, ipynb: str):
#         cells = json.load(open(ipynb))
#         cnt = 1
#         clean_version = []
#         for cell in cells["cells"]:
#             if cell["cell_type"] == "code":
#                 if not cell["source"]:
#                     continue
#                 if not cell["source"][0].startswith("#"):
#                     continue
#                 cell["execution_count"] = None
#             elif cell["cell_type"] == "markdown":
#                 if not cell["source"]:
#                     continue
#                 headline = cell["source"][0]
#                 if headline.startswith("#") or headline.startswith("<a id="):
#                     if cnt == 1:
#                         print("# TOC")
#                     header = headline.lstrip("# ").strip()
#                     print(f"* [{header}](#cell{cnt})")
#                     cell["source"].insert(1, f'<a id="cell{cnt}"></a>\n')
#                     cnt += 1
#             clean_version.append(cell)
#         out_file = open(ipynb, "w")
#         cells["cells"] = clean_version
#         json.dump(cells, out_file, indent=2)
#         out_file.close()

In [1]:
# !nbblack 00_core.ipynb
!nbdev_build_lib #--fname 04_misc.ipynb
# !nbdev_build_docs

Converted 00_core.ipynb.
Converted 01_viz.ipynb.
Converted 02_snip.ipynb.
Converted 03_ver.ipynb.
Converted 04_misc.ipynb.
Converted index.ipynb.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext sniptool